In [ ]:
import bs4
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
import itertools
import os
import zipfile
from pathlib import Path 
import itertools

In [ ]:
def extract(f):
  file=open(f)
  contents=file.read()
  soup = BeautifulSoup(contents,features="xml")
  cont = soup.findAll('r')
  text1 = []
  param1=[]
  data={}
  for x in cont: 
    for y in x:
        if isinstance(y, bs4.element.NavigableString):
            text1.append(y.strip())
            z="".join(text1)
            param1.append(z)
    text1=[]
    z=""
  for i,r in enumerate(soup.find_all('r')):
    data[i]=r.attrs
    for m in r.findChildren():
        data[i]['m']=list(m.attrs.values())
  df1=pd.DataFrame.from_dict(data, orient='index')
  cont1 = soup.findAll('measType')
  text = []
  param=[]
  for x in cont1:
    for y in x:
        if isinstance(y, bs4.element.NavigableString):
            text.append(y.strip())
            z="".join(text)
            param.append(z)
    text=[]
    z=""
  p_val= list(df1['p'])
  time_stamp=[]
  final_time=[]
  cell=[]
  final_cell=[]
  for time in soup.findAll('measValue'):
    stamp = time['measObjLdn']
    new_stamp=stamp[31:]
    new_cell=stamp[5:18]
    time_stamp.append(new_stamp)
    cell.append(new_cell)
    l=int(len(p_val)/len(time_stamp))
  for i in time_stamp:
    for j in range(0,l):
        final_time.append(i)
  for i in cell:
    for j in range(0,l):
        final_cell.append(i)
  param0=[]
  x=int(len(p_val)/len(param))
  for i in range(0,x):
    for j in param:
      param0.append(j)
  final_df= pd.DataFrame(param0)
  final_df['R value'] = param1
  final_df['P']=p_val
  final_df['Start Time'] = final_time
  final_df['Cell ID'] = final_cell
  final_df.columns=['Parameter','P','R Value','Start Time','Cell ID']
  for i in soup.findAll('fileSender'):
    localDn=i['localDn']
    enb=localDn[17:21]
    ip=localDn[38:51]
    title=localDn[52:]
  for i in soup.findAll('managedElement'):
    ver=i['swVersion']
  n=len(p_val)
  ENB=itertools.repeat(enb,n)
  IP=itertools.repeat(ip,n)
  TITLE=itertools.repeat(title,n)
  VER=itertools.repeat(ver,n)
  new_df=pd.DataFrame({'ENB':ENB,'IP': IP,'TITLE': TITLE, 'VERSION': VER})
  combined_df=pd.concat([final_df,new_df], axis=1)
  combined_df=combined_df.drop(['P'], axis = 1)
  return combined_df
  


In [ ]:
zip1 = zipfile.ZipFile('AB-2521-11-0063.zip')
list1= zip1.namelist()
with zipfile.ZipFile('AB-2521-11-0063.zip',"r") as zip_ref:
    z1=zip_ref.extractall()

In [ ]:
final_list=[]
for i in list1:
  if i.endswith('xml'):
    final_list.append(i)
print(final_list)


['AB-2521-11-0063/2000/20000105/C20000105.0600-20000105.0700_0495-AB-2521-11-0063.xml', 'AB-2521-11-0063/2000/20000105/C20000105.0700-20000105.0800_0495-AB-2521-11-0063.xml', 'AB-2521-11-0063/2000/20000105/C20000105.0800-20000105.0900_0495-AB-2521-11-0063.xml', 'AB-2521-11-0063/2000/20000105/C20000105.0900-20000105.1000_0495-AB-2521-11-0063.xml', 'AB-2521-11-0063/2000/20000105/C20000105.1000-20000105.1100_0495-AB-2521-11-0063.xml', 'AB-2521-11-0063/2000/20000105/C20000105.1100-20000105.1200_0495-AB-2521-11-0063.xml', 'AB-2521-11-0063/2000/20000105/C20000105.1200-20000105.1300_0495-AB-2521-11-0063.xml', 'AB-2521-11-0063/2000/20000105/C20000105.1300-20000105.1400_0495-AB-2521-11-0063.xml', 'AB-2521-11-0063/2000/20000105/C20000105.1400-20000105.1500_0495-AB-2521-11-0063.xml', 'AB-2521-11-0063/2000/20000105/C20000105.1500-20000105.1600_0495-AB-2521-11-0063.xml', 'AB-2521-11-0063/2000/20000105/C20000105.1600-20000105.1700_0495-AB-2521-11-0063.xml', 'AB-2521-11-0063/2000/20000105/C20000105.1

In [ ]:
l=[]
for i in final_list:
  try:
     l.append(extract(i))
  except Exception:
     pass

['RRC.ConnEstabAtt.Emergency', 'RRC.ConnEstabAtt.HighPriorityAccess', 'RRC.ConnEstabAtt.MtAccess', 'RRC.ConnEstabAtt.MoSignalling', 'RRC.ConnEstabAtt.MoData', 'RRC.ConnEstabSucc.Emergency', 'RRC.ConnEstabSucc.HighPriorityAccess', 'RRC.ConnEstabSucc.MtAccess', 'RRC.ConnEstabSucc.MoSignalling', 'RRC.ConnEstabSucc.MoData', 'RRC.ConnEstabFaileNBCause.Congestion', 'RRC.ConnEstabFaileNBCause.Unspecified', 'RRC.ConnEstabFaileNBCause.EnergySaving', 'RRC.ConnReEstabAtt.ReconfigurationFailure', 'RRC.ConnReEstabAtt.HandoverFailure', 'RRC.ConnReEstabAtt.OtherFailure', 'RRC.ConnReEstabAtt.Sum', 'RRC.ConnReEstabSucc.ReconfigurationFailure', 'RRC.ConnReEstabSucc.HandoverFailure', 'RRC.ConnReEstabSucc.OtherFailure', 'RRC.ConnReEstabSucc.Sum', 'RRC.ConnReEstabFail.ReconfigurationFailure', 'RRC.ConnReEstabFail.HandoverFailure', 'RRC.ConnReEstabFail.OtherFailure', 'RRC.ConnReEstabFail.Sum', 'RRC.ConnMean', 'RRC.ConnMax', 'RRC.ConnEstabTimeMean.Emergency', 'RRC.ConnEstabTimeMean.HighPriorityAccess', 'RRC.

KeyboardInterrupt: ignored

In [ ]:
new_df = pd.concat(l)
print(new_df)
new_excel=new_df.to_excel('Combined_Excel1.xlsx')